# Módulo 2: Scraping con Selenium
## LATAM Airlines
<div align="center">
<a href="https://www.latam.com/"><img src="https://i.pinimg.com/originals/dd/52/74/dd5274702d1382d696caeb6e0f6980c5.png"  width="420"></img></a>
<br>
</div>

Vamos a scrapear el sitio de Latam para averiguar datos de vuelos en funcion el origen y destino, fecha y cabina. La información que esperamos obtener de cada vuelo es:
- Precio(s) disponibles
- Horas de salida y llegada (duración)
- Información de las escalas

**¡Empecemos!**
Utilicemos lo aprendido hasta ahora para lograr el objetivo propuesto

In [ ]:
import requests
from bs4 import BeautifulSoup

In [ ]:
url = 'https://www.latam.com/es_mx/apps/personas/booking?fecha1_dia=10&fecha1_anomes=2020-11&auAvailability=1&ida_vuelta=ida&vuelos_origen=Ciudad%20de%20M%C3%A9xico&from_city1=MEX&vuelos_destino=Santiago%20de%20Chile&to_city1=SCL&flex=1&vuelos_fecha_salida_ddmmaaaa=10/11/2020&cabina=Y&nadults=1&nchildren=0&ninfants=0&cod_promo=&stopover_outbound_days=0&stopover_inbound_days=0#/'

In [ ]:
r = requests.get(url)

In [ ]:
r.status_code

In [ ]:
s = BeautifulSoup(r.text, 'lxml')

In [ ]:
print(s.prettify())

Vemos que la respuesta de la página no contiene la información que buscamos, ya que la misma aparece recién después de ejecutar el código JavaSCript que está en la respuesta.

## Selenium

Selenium es una herramienta que nos permitirá controlar un navegador y podremos utilizar las funcionalidades del motor de JavaScript para cargar el contenido que no viene en el HTML de la página. Para esto necesitamos el módulo `webdriver`.

In [ ]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

Paso 1: instanciar un **driver** del navegador

In [ ]:
options = webdriver.ChromeOptions()
options.add_argument('--incognito')
driver = webdriver.Chrome(ChromeDriverManager().install(), options=options)

Paso 2: hacer que el navegador cargue la página web.

In [ ]:
driver.get(url)

Paso 3: extraer la información de la página

In [ ]:
vuelos = driver.find_elements_by_xpath('//li[@class="flight"]')
vuelos

In [ ]:
vuelo = vuelos[1]

In [ ]:
vuelo

In [ ]:
#salida
salida = vuelo.find_element_by_xpath('.//div[@class="departure"]/time').get_attribute('datetime')
# Llegada
llegada = vuelo.find_element_by_xpath('.//div[@class="arrival"]/time').get_attribute('datetime')
# Duración
duracion = vuelo.find_element_by_xpath('.//span[@class="duration"]/time').get_attribute('datetime')

In [ ]:
salida

In [ ]:
llegada

In [ ]:
duracion

In [ ]:
boton_escalas = vuelo.find_element_by_xpath('.//div[@class="flight-summary-stops-description"]/button')

In [ ]:
boton_escalas.click()

In [ ]:
segmentos = vuelo.find_elements_by_xpath('//div[@class="sc-cLQEGU hyoued"]/div[@class="sc-esOvli iQSbOQ"]')

In [ ]:
segmentos

In [ ]:
escalas = len(segmentos) - 1
escalas

In [ ]:
segemento = segmentos[0]

In [ ]:
segemento.find_element_by_xpath('.//div[@class="sc-bwCtUz iybVbT"]/abbr[@class="sc-hrWEMg hlCkST"]').text

In [ ]:
segemento.find_element_by_xpath('.//div[@class="sc-bwCtUz iybVbT"]/time[@class="sc-RefOD libzvk"]').text

In [ ]:
driver.find_element_by_xpath('//div[@class="modal-header sc-dnqmqq cGfTsx"]//button[@class="close"]').click()

In [ ]:
vuelo.click()

In [ ]:
tarifas = vuelo.find_elements_by_xpath('.//div[@class="fares-table-container"]//tfoot//td[contains(@class, "fare-")]')

In [ ]:
tarifas

In [ ]:
precios = []
for tarifa in tarifas:
    nombre = tarifa.find_element_by_xpath('.//label').get_attribute('for')
    moneda = tarifa.find_element_by_xpath('.//span[@class="price"]/span[@class="currency-symbol"]').text
    valor = tarifa.find_element_by_xpath('.//span[@class="price"]/span[@class="value"]').text 
    dict_tarifa={nombre:{'moneda':moneda, 'valor':valor}}
    precios.append(dict_tarifa)
    print(dict_tarifa)

In [ ]:
def obtener_precios(vuelo):
    '''
    Función quye retorna una lista de diccionarios con las distintas tarifas
    '''
    tarifas=vuelo.find_elements_by_xpath('.//div[@class="fares-table-container"]//tfoot//td[contains(@class, "fare-")]')
    precios=[]
    for tarifa in tarifas:
        nombre=tarifa.find_element_by_xpath('.//label').get_attribute('for')
        moneda=tarifa.find_element_by_xpath('.//span[@class="price"]/span[@class="currency-symbol"]').text
        valor =tarifa.find_element_by_xpath('.//span[@class="price"]/span[@class="value"]').text
        dict_tarifa={nombre:{'moneda':moneda,'valor':valor}}
        precios.append(dict_tarifa)
        print(dict_tarifa)
    return precios


def obtener_datos_escalas(vuelo):
    '''
    Función  que retorna una lista de diccionarios 
    con las información de las escalas de cada vuelo
    '''
    segmentos=vuelo.find_elements_by_xpath('//div[@class="sc-hZSUBg bOhNnO"]/div[@class="sc-cLQEGU hyoued"]')
    info_escalas = []
    for segmento in segmentos:
        origen=segmento.find_elements_by_xpath('.//div[@class="sc-iujRgT jEtESl"]//abbr[@class="sc-hrWEMg hlCkST"]')[0].text
        hra_salida=segmento.find_elements_by_xpath('.//div[@class="sc-iujRgT jEtESl"]//time')[0].get_attribute('datetime')
        destino=segmento.find_elements_by_xpath('.//div[@class="sc-iujRgT jEtESl"]//abbr[@class="sc-hrWEMg hlCkST"]')[1].text
        hra_llegad=segmento.find_elements_by_xpath('.//div[@class="sc-iujRgT jEtESl"]//time')[1].get_attribute('datetime')
        duracion=segmento.find_element_by_xpath('.//span[@class="sc-cmthru ipcOEH"]//time').get_attribute('datetime')
        num_vuelo=segmento.find_element_by_xpath('.//div[@class="sc-hMFtBS dhxqHN"]//b').text
        mod_avion=segmento.find_element_by_xpath('.//div[@class="sc-hMFtBS dhxqHN"]//span[@class="sc-gzOgki uTyOl"]').text
        
        data_dict={
            'origen':origen,
            'hr_salida':hra_salida,
            'destino':destino,
            'hra-llegada':hra_salida,
            'duracion':duracion,
            'num_vuelo':num_vuelo,
            'mod_avion':mod_avion
        }
        info_escalas.append(data_dict)
        
    return info_escalas


def obtener_tiempos(vuelo):
    '''
    Función que retorna un diccionario con los horarios de salida y llegada de cada
    vuelo, incluyendo la duración. 
    Nota: la duración del vuelo no es la hora de llegada - la hora de salida porque
    puede haber diferencia de horarios entre el origen y el destino.
    '''
    # Hora de salida
    salida = vuelo.find_element_by_xpath('.//div[@class="departure"]/time').get_attribute('datetime')
    
    # Hora de llegada
    llegada =  vuelo.find_element_by_xpath('.//div[@class="arrival"]/time').get_attribute('datetime')
    
    # Duracion
    duracion = vuelo.find_element_by_xpath('.//span[@class="duration"]/time').get_attribute('datetime')
    
    tiempos = {'hora_salida': salida, 'hora_llegada': llegada, 'duracion': duracion}
    return tiempos


def obtener_info(driver):
    #cerrar ventana emergente innecesaria que impide siguiente paso
    driver.find_element_by_xpath('//div[@class="tooltip-container sc-kUaPvJ ikSqYE"]//button[@class="close"]').click()
    
    
    vuelos=driver.find_elements_by_xpath('//li[@class="flight"]')
    print(f'Se encontraron{len(vuelos)} vuelos.')
    print('iniciando Scraping...')
    info=[]
    
    for vuelo in vuelos:
        #obtener tiempo generales
        tiempos=obtener_tiempos(vuelo)
        #clickeamos sobre el b oton de las escalas
        vuelo.find_element_by_xpath('.//div[@class="flight-summary-stops-description"]/button').click()
        escalas=obtener_datos_escalas(vuelo)
        #cerramos el modelos
        driver.find_element_by_xpath('//div[@class="modal-content sc-iwsKbI eHVGAN"]//button[@class="close"]').click()
        #clickeamos el vuelo para ver los precios
        vuelo.click()
        precios=obtener_precios(vuelo)
        vuelo.click()
        info.append({'precios':precios,'tiempo':tiempos, 'escalas':escalas})
    return info

In [ ]:
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
options = webdriver.ChromeOptions()
options.add_argument('--incognito')
driver = webdriver.Chrome(ChromeDriverManager().install(), options=options)
driver.get(url)

In [ ]:
# Introducir una demora
delay = 10
try:
    # introducir demora inteligente
    vuelo = WebDriverWait(driver, delay).until(EC.presence_of_element_located((By.XPATH, '//li[@class="flight"]')))
    print('La página terminó de cargar')
    info_vuelos = obtener_info(driver)
except TimeoutException:
    print('La página tardó demasiado en cargar')
    info_vuelos = []
driver.close()
info_vuelos

Paso 4: cerrar el navegador

In [ ]:
driver.close()